<a href="https://colab.research.google.com/github/Dlogical23/MedInputClass/blob/main/MedInputClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Assuming your CSV file is located at '/content/drive/My Drive/your_file.csv'
# Replace 'your_file.csv' with the actual path to your CSV file
file_path = '/content/drive/MyDrive/overview-of-recordings.csv'
try:
  df = pd.read_csv(file_path)
  print("Successfully imported CSV file.")
  # Now you can work with the DataFrame 'df'
  print(df.head()) # Display first few rows of the DataFrame
except FileNotFoundError:
  print(f"Error: File not found at {file_path}. Please check the file path.")
except pd.errors.ParserError:
  print(f"Error: Unable to parse the CSV file at {file_path}. Please check the file format.")
except Exception as e:
  print(f"An unexpected error occurred: {e}")

Successfully imported CSV file.
   audio_clipping  audio_clipping:confidence background_noise_audible  \
0     no_clipping                     1.0000              light_noise   
1  light_clipping                     0.6803                 no_noise   
2     no_clipping                     1.0000                 no_noise   
3     no_clipping                     1.0000              light_noise   
4     no_clipping                     1.0000                 no_noise   

   background_noise_audible:confidence  overall_quality_of_the_audio  \
0                               1.0000                          3.33   
1                               0.6803                          3.33   
2                               0.6655                          3.33   
3                               1.0000                          3.33   
4                               1.0000                          4.67   

     quiet_speaker  quiet_speaker:confidence  speaker_id  \
0  audible_speaker                  

In [19]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch

# Extract texts (medical transcriptions)
texts = df["phrase"].tolist()

# Extract labels (diagnosis codes or symptoms)
labels = df["prompt"].tolist()

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=25)  # Adjust num_labels

# Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

# Convert to PyTorch datasets
class MedicalDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MedicalDataset(train_encodings, train_labels)
test_dataset = MedicalDataset(test_encodings, test_labels)
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch

# Load dataset (replace with your medical transcription dataset)
texts = [...]  # List of medical transcriptions
labels = [...]  # Corresponding labels (e.g., diagnosis codes)

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=25)  # Adjust num_labels

# Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

# Convert to PyTorch datasets
class MedicalDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MedicalDataset(train_encodings, train_labels)
test_dataset = MedicalDataset(test_encodings, test_labels)

# Training setup (continued)
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Loss: {avg_train_loss}")

    # Evaluation after each epoch
    model.eval()
    total_eval_accuracy = 0
    for batch in test_loader:
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)

            total_eval_accuracy += accuracy_score(labels.cpu(), predictions.cpu())

    avg_eval_accuracy = total_eval_accuracy / len(test_loader)
    print(f"Epoch {epoch + 1}, Accuracy: {avg_eval_accuracy}")

# Save the model, tokenizer, and label mappings
model.save_pretrained("BertModelAndTokenizer")
tokenizer.save_pretrained("BertModelAndTokenizer")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Loss: 3.5872604846954346
Epoch 1, Accuracy: 0.0
Epoch 2, Loss: 3.3486216068267822
Epoch 2, Accuracy: 0.0
Epoch 3, Loss: 3.1659557819366455
Epoch 3, Accuracy: 0.0
Epoch 4, Loss: 3.1196954250335693
Epoch 4, Accuracy: 0.0
Epoch 5, Loss: 3.1146955490112305
Epoch 5, Accuracy: 0.0


('BertModelAndTokenizer/tokenizer_config.json',
 'BertModelAndTokenizer/special_tokens_map.json',
 'BertModelAndTokenizer/vocab.txt',
 'BertModelAndTokenizer/added_tokens.json')

In [20]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

# Load the saved model and tokenizer
model = BertForSequenceClassification.from_pretrained("BertModelAndTokenizer")
tokenizer = BertTokenizer.from_pretrained("BertModelAndTokenizer")

# Move the model to the appropriate device (CPU or GPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.eval()  # Set the model to evaluation mode

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [21]:
# Example new medical transcription data
new_texts = [
    "Patient presents with chest pain and shortness of breath.",
    "History of diabetes and hypertension."
]

# Tokenize the new data
new_encodings = tokenizer(new_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")

# Move the tokenized data to the same device as the model
input_ids = new_encodings['input_ids'].to(device)
attention_mask = new_encodings['attention_mask'].to(device)

In [24]:
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

# Convert predictions to labels (if you have label mappings)
# Update label_mappings to include all possible labels (0-24)
label_mappings = {i: f"Condition {i}" for i in range(25)}
predicted_labels = [label_mappings[pred.item()] for pred in predictions]

# Print the results
for text, label in zip(new_texts, predicted_labels):
    print(f"Text: {text}\nPredicted Label: {label}\n")

Text: Patient presents with chest pain and shortness of breath.
Predicted Label: Condition 18

Text: History of diabetes and hypertension.
Predicted Label: Condition 18



DEPLOYMENT

In [27]:
!pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.5 MB/s eta 0:00:00


In [28]:
from fastapi import FastAPI, Request
import torch
from transformers import BertForSequenceClassification, BertTokenizer

app = FastAPI()

# Load the model and tokenizer
model = BertForSequenceClassification.from_pretrained("BertModelAndTokenizer")
tokenizer = BertTokenizer.from_pretrained("BertModelAndTokenizer")
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.eval()

@app.post("/predict")
async def predict(request: Request):
    data = await request.json()
    texts = data.get("texts", [])

    # Tokenize the input texts
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=512, return_tensors="pt")
    input_ids = encodings['input_ids'].to(device)
    attention_mask = encodings['attention_mask'].to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
from fastapi import FastAPI, Request
import torch
from transformers import BertForSequenceClassification, BertTokenizer

app = FastAPI()

# Load the model and tokenizer
model = BertForSequenceClassification.from_pretrained("BertModelAndTokenizer")
tokenizer = BertTokenizer.from_pretrained("BertModelAndTokenizer")
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.eval()

@app.post("/predict")
async def predict(request: Request):
    data = await request.json()
    texts = data.get("texts", [])

    # Tokenize the input texts
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=512, return_tensors="pt")
    input_ids = encodings['input_ids'].to(device)
    attention_mask = encodings['attention_mask'].to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

    # Convert predictions to labels
    label_mappings = {0: "Condition A", 1: "Condition B", 2: "Condition C"}  # Replace with your actual label mappings
    predicted_labels = [label_mappings.get(pred.item(), "Unknown") for pred in predictions]

    return {"predictions": predicted_labels}

# Run the API
# Use `uvicorn <filename>:app --reload` to start the server
    return {"predictions": predicted_labels}

# Run the API
# Use `uvicorn <filename>:app --reload` to start the server

FINE-TUNING

In [35]:
# Assuming 'df' is your DataFrame from the CSV file
texts = df["phrase"].tolist()
labels = df["prompt"].tolist()

# Create a mapping from labels to numerical IDs
unique_labels = list(set(labels))
label_to_id = {label: i for i, label in enumerate(unique_labels)}

# Convert labels to numerical IDs using the mapping
labels = [label_to_id[label] for label in labels]

# Now perform the train_test_split
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    texts, labels, test_size=0.3, random_state=42
)

In [36]:
from torch.utils.data import DataLoader

# Split data into train, validation, and test sets
train_texts, temp_texts, train_labels, temp_labels = train_test_split(texts, labels, test_size=0.3, random_state=42)  # Adjust test_size as needed
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5, random_state=42) # Splitting temp into val and tesT

# Tokenize data for all three sets
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)  # Tokenize validation data
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

# Create Datasets for all three sets
train_dataset = MedicalDataset(train_encodings, train_labels)
val_dataset = MedicalDataset(val_encodings, val_labels)  # Create validation dataset
test_dataset = MedicalDataset(test_encodings, test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

epochs = 3  # Adjust based on performance
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Loss: {avg_train_loss}")

    # Evaluate on validation set
    model.eval()
    total_eval_accuracy = 0
    for batch in val_loader:
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)

            total_eval_accuracy += accuracy_score(labels.cpu(), predictions.cpu())

    avg_eval_accuracy = total_eval_accuracy / len(val_loader)
    print(f"Epoch {epoch + 1}, Validation Accuracy: {avg_eval_accuracy}")

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Loss: 2.688893748064564
Epoch 1, Validation Accuracy: 0.8042800453514739
Epoch 2, Loss: 1.194597749269172
Epoch 2, Validation Accuracy: 0.9573412698412699
Epoch 3, Loss: 0.465682319429231
Epoch 3, Validation Accuracy: 0.9831349206349206
